In [ ]:
import requests
import json

params = {
    "roster": "FA25",   # e.g., Spring 2024
    "subject": "CS"     # e.g., Computer Science
}
r = requests.get("https://classes.cornell.edu/api/2.0/search/classes.json", params=params)
data = r.json()["data"]["classes"]

classes = []
for c in data:
    subject = c.get("subject")
    catalog_nbr = c.get("catalogNbr")
    title = c.get("titleLong") or c.get("titleShort")
    description = c.get("description")
    attrs = c.get("crseAttrs", [])
    attr_groups = c.get("crseAttrValueGroups", [])

    # 📌 handle prereqs/coreqs (robust across rosters)
    prereq = c.get("catalogPrereq")
    coreq = c.get("catalogCoreq")
    legacy_prereq_coreq = c.get("catalogPrereqCoreq")

    # sections / meetings / instructors
    sections = []
    for eg in c.get("enrollGroups", []):
        for sec in eg.get("classSections", []):
            for mtg in sec.get("meetings", []):
                instructors = [
                    i.get("name") for i in mtg.get("instructors", [])
                    if i.get("name")
                ]
                sections.append({
                    "section": sec.get("classNbr"),
                    "component": sec.get("ssrComponent"),
                    "pattern": mtg.get("pattern"),
                    "facility": mtg.get("facilityDescr"),
                    "start": mtg.get("timeStart"),
                    "end": mtg.get("timeEnd"),
                    "instructors": instructors
                })

    classes.append({
        "subject": subject,
        "catalog_nbr": catalog_nbr,
        "title": title,
        # "description": description,
        "course_attributes": attrs,
        "attribute_groups": attr_groups,
        "prereq": prereq,
        "coreq": coreq,
        "legacy_prereq_coreq": legacy_prereq_coreq
    })

print(classes[:2])

# after your loop, once classes[] is built:
with open("classes.json", "w", encoding="utf-8") as f:
    json.dump(classes, f, indent=2, ensure_ascii=False)


[{'subject': 'CS', 'catalog_nbr': '1110', 'title': 'Introduction to Computing: A Design and Development Perspective', 'description': 'Programming and problem solving using Python. Emphasizes principles of software development, style, and testing. Topics include procedures and functions, iteration, recursion, arrays and vectors, strings, an operational model of procedure and function calls, algorithms, exceptions, object-oriented programming. Weekly labs provide guided practice on the computer, with staff present to help.', 'course_attributes': [{'crseAttr': '0012', 'crseAttrValue': 'SMR-AS', 'descr': 'AS-Symbolic & Mathematic Rsng', 'descrformal': 'AS-Symbolic and Mathematical Reasoning', 'attrDescr': 'Distribution Requirements', 'attrDescrShort': 'DistReq'}, {'crseAttr': '0012', 'crseAttrValue': 'STA-IL', 'descr': 'IL-Statistics Requirement', 'descrformal': 'IL-Statistics Requirement', 'attrDescr': 'Distribution Requirements', 'attrDescrShort': 'DistReq'}], 'attribute_groups': [{'attr